# LEEP 2024 Multi-Country CEA

In [384]:
import numpy as np
import pandas as pd
import squigglepy as sq
# from squigglepy.numbers import K

pd.set_option('display.float_format', lambda x: f'{x:,.3f}')
idx = pd.IndexSlice

sq.set_seed(42)

COUNTRIES = ["Malawi", "Madagascar", "Sierra Leone", "Liberia", "Angola", "Pakistan",
             "Niger", "Zimbabwe", "Bolivia", "Ghana", "Nigeria", "Cote d'Ivoire", "Senegal"]
SCENARIOS = ["Counterfactual", "LEEP intervention"]
YEARS = range(2021, 2101)

# N = 100*K
N = 10
SAMPLES = range(N)

SAMPLE_YEAR_INDEX = pd.MultiIndex.from_product(
    [SAMPLES, YEARS], names=["Sample", "Year"])
SCENARIO_SAMPLE_INDEX = pd.MultiIndex.from_product(
    [SCENARIOS, SAMPLES], names=["Scenario", "Sample"])
SCENARIO_SAMPLE_YEAR_INDEX = pd.MultiIndex.from_product(
    [SCENARIOS, SAMPLES, YEARS], names=["Scenario", "Sample", "Year"])

## Inputs

### Population

The population and projected future population estimates for each country are from the [UN population projections](https://population.un.org/wpp/Download/Standard/CSV/).

In [385]:
# TODO(drw): Could use uncertainty estimates here?
population = pd.read_csv('population.csv', index_col=0)
population.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
Year,,,,,,,,,,,,,
2021,19889742,28915652,8420642,5193422,34503776,231402110,25252722,15993525,12079474,32833036,213401330,27478250,16876726
2022,20405318,29611718,8605723,5302690,35588996,235824860,26207982,16320539,12224114,33475870,218541220,28160548,17316452
2023,20931750,30325736,8791092,5418378,36684212,240485660,27202844,16665407,12388575,34121990,223804640,28873040,17763164
2024,21475962,31056608,8977979,5536954,37804630,245209810,28238976,17020320,12567344,34777524,229152220,29603304,18221560
2025,22033992,31797310,9165381,5657236,38942796,249948880,29315974,17375286,12746151,35439790,234573600,30344186,18687806


### Births

Projections of future annual births in each country are from the [UN population projections](https://population.un.org/wpp/Download/Standard/Population/).

In [386]:
# TODO(drw): Could use uncertainty estimates here?
births = pd.read_csv('births.csv', index_col=0)
births.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
Year,,,,,,,,,,,,,
2021,653703,895045,264106,162685,1338792,6374741,1144371,488807,263820,904679,7923294,932943,549984
2022,665678,906130,265023,164452,1359723,6424944,1180654,491195,264070,906677,8002758,943133,555696
2023,678962,918152,266774,166979,1381742,6474370,1216927,493840,263936,909913,8109136,959962,562482
2024,687950,927190,267219,169153,1404185,6512176,1256001,496655,265235,911299,8234399,974998,568369
2025,698306,937701,268121,171399,1426882,6559877,1296167,498207,266191,916924,8321052,989981,575820


### Decline in lead market share once enforcement/compliance begins

We estimate that, once enforcement/compliance begins, the market share captured by lead-based paint will decline at a rate of 20% annually. This is a conservative estimate relative to our compliance goal: in the majority of our target countries, we aim to achieve 50% compliance a year after regulation is implemented and enforcement begins. [Follow up study](https://leadelimination.org/malawi-follow-up-study/) data from Malawi suggests that our compliance goals are achievable, as do preliminary reports of manufacturers representing >50% of market share switching to lead-free alternatives in Pakistan, Ghana, and Zimbabwe.

In [387]:
decline_in_lead_market_share_once_enforcement_compliance_begins = pd.Series(
    sq.norm(-0.3, -0.1) @ N)
decline_in_lead_market_share_once_enforcement_compliance_begins.head()

0   -0.181
1   -0.263
2   -0.154
3   -0.143
4   -0.319
dtype: float64

### % of new painted homes that use some colored solvent based lead paint

To develop this estimate, we surveyed members of the LEEP team to understand their best guess of the portion of homes that use some coloured solvent-based paints in the countries where they work. We averaged the estimates provided by LEEP’s team members, which gave us the estimate that coloured solvent-based paint is used in 80% of new painted homes in the LMICs included in this model. We are uncertain of this estimate given the difficulty of approximating this input across the variety of painting practices used in LMICs.

In [388]:
pct_of_new_painted_homes_that_use_some_colored_solvent_based_lead_paint = pd.Series(
    sq.lognorm(0.65, 0.9) @ N)
pct_of_new_painted_homes_that_use_some_colored_solvent_based_lead_paint.head()

0   0.834
1   0.826
2   0.770
3   0.855
4   0.801
dtype: float64

### Lead market share of colored solvent based paints (2021)

In [389]:
lead_market_share_of_colored_solvent_based_paints_2021 = pd.DataFrame(
    np.array(
        [sq.lognorm(0.60, 0.83) @ N,
         sq.lognorm(0.60, 0.83) @ N,
         sq.lognorm(0.32, 0.44) @ N,
         sq.lognorm(0.60, 0.83) @ N,
         sq.lognorm(0.48, 0.66) @ N,
         sq.lognorm(0.52, 0.72) @ N,
         sq.lognorm(0.32, 0.44) @ N,
         sq.lognorm(0.52, 0.72) @ N,
         sq.lognorm(0.32, 0.44) @ N,
         sq.lognorm(0.16, 0.22) @ N,
         sq.lognorm(0.68, 0.94) @ N,
         sq.lognorm(0.60, 0.83) @ N,
         sq.lognorm(0.60, 0.83) @ N]).T,
    columns=COUNTRIES)
lead_market_share_of_colored_solvent_based_paints_2021.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
0,0.693,0.872,0.403,0.726,0.478,0.539,0.392,0.654,0.362,0.196,0.723,0.679,0.735
1,0.660,0.678,0.395,0.751,0.545,0.547,0.352,0.606,0.426,0.192,0.814,0.616,0.640
2,0.796,0.671,0.352,0.611,0.572,0.559,0.362,0.587,0.363,0.177,0.817,0.751,0.572
3,0.695,0.651,0.384,0.684,0.596,0.643,0.362,0.607,0.403,0.163,0.914,0.690,0.725
4,0.676,0.750,0.379,0.674,0.603,0.621,0.334,0.518,0.343,0.189,0.868,0.610,0.651


### Annual time discount rate

We discount future costs and benefits at a rate of 4% per year, in line with [GiveWell](https://docs.google.com/spreadsheets/d/1B1fODKVbnGP4fejsZCVNvBm5zvI1jC7DhkaJpFk6zfo/edit#gid=1362437801). There is debate over the appropriate discount rate. [Haacker, Hallett, and Atun (2020)](https://academic.oup.com/heapol/article/35/1/107/5591528) found that 85% of 188 global health CEAs used a discount of 3% for both health and costs. Given the 100-year timeframe of this model, this is an important factor (e.g. a value of 1 in year 1 discounted by 4% per year would be 0.46 in year 20, 0.14 in year 50, and 0.06 by year 70; whereas using a discount rate of 3% would give a value of 0.55 in year 20, 0.23 in year 50, and 0.13 by year 70).

In [390]:
annual_time_discount_rate = pd.Series(sq.norm(0.03, 0.05) @ N)
annual_time_discount_rate.head()

0   0.039
1   0.034
2   0.030
3   0.037
4   0.040
dtype: float64

### Discount factor

In [391]:
discount_factor = pd.Series(
    np.array([1/(1 + annual_time_discount_rate)**(year - YEARS[0])
             for year in YEARS]).T.flatten(),
    index=SAMPLE_YEAR_INDEX)
discount_factor.head()

Sample  Year
0       2021   1.000
        2022   0.962
        2023   0.926
        2024   0.891
        2025   0.858
dtype: float64

In [392]:
SAMPLE_YEAR_INDEX.shape

(800,)

### Increase in BLL from living in a home with lead paint

In [393]:
increase_in_bll_from_living_in_a_home_with_lead_paint = pd.Series(
    sq.norm(0.7, 3.0) @ N)
increase_in_bll_from_living_in_a_home_with_lead_paint.head()

0   1.175
1   1.343
2   3.338
3   1.276
4   2.436
dtype: float64

### Implied % of childhood BLL caused by lead paint initially

In [394]:
# TODO(drw): This formula references non-inputs

### Probability of success

We calculate a probability of success for each country, which encapsulates our estimate of the likelihood that our intervention that country is likely to succeed in its implementation, and hence that its associated benefits will be realised. These figures were estimated by the LEEP team based on knowledge of each country and an assessment of current progress towards implementation in each country. Overall, our probability of success estimates are relatively high, since they mostly capture the “stochastic” possibility of failure due to events such as unexplained government withdrawal / reversal, or political instability, which we feel to be relatively unlikely. The possibility of implementation being delayed, on the other hand, is already captured in our uncertainty intervals around intervention start dates.

We multiply total discounted benefits by these probabilities of success to get total expected discounted benefits, and calculate the final cost-effectiveness number as total discounted costs divided by total expected discounted benefits. We choose only to weight benefits, and not costs, by the probabilities of success, in order to be conservative and reflect the possibility that failure occurs even despite government and LEEP costs having already been incurred.


In [395]:
probability_of_success = pd.DataFrame(
    np.array(
        [sq.lognorm(0.85, 0.95) @ N,
         sq.lognorm(0.75, 0.85) @ N,
         sq.lognorm(0.70, 0.80) @ N,
         sq.lognorm(0.65, 0.75) @ N,
         sq.lognorm(0.75, 0.85) @ N,
         sq.lognorm(0.85, 0.95) @ N,
         sq.lognorm(0.70, 0.80) @ N,
         sq.lognorm(0.75, 0.85) @ N,
         sq.lognorm(0.70, 0.80) @ N,
         sq.lognorm(0.75, 0.85) @ N,
         sq.lognorm(0.75, 0.85) @ N,
         sq.lognorm(0.70, 0.80) @ N,
         sq.lognorm(0.70, 0.80) @ N,]).T,
    columns=COUNTRIES)

probability_of_success.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
0,0.879,0.839,0.803,0.709,0.827,0.892,0.705,0.772,0.737,0.791,0.777,0.763,0.741
1,0.912,0.805,0.743,0.742,0.759,0.906,0.731,0.796,0.759,0.790,0.808,0.697,0.753
2,0.885,0.786,0.737,0.701,0.799,0.904,0.758,0.747,0.803,0.806,0.795,0.777,0.794
3,0.862,0.833,0.794,0.718,0.784,0.866,0.786,0.755,0.719,0.782,0.865,0.762,0.674
4,0.861,0.812,0.715,0.639,0.789,0.901,0.727,0.866,0.741,0.813,0.848,0.715,0.741


## Births with averted lead paint exposure

### Number of homes painted

#### People per household

We use recent data on average household size from the [Global Data Lab](https://globaldatalab.org/areadata/table/hhsize/AGO+CIV+GHA+LBR+MDG+MWI+NER+NGA+SEN+SLE+ZWE+BOL+PAK/?levels=1+4). We assume that average household size will decrease at a constant rate of 1% annually across countries. Average household size in high income countries is approximately 2.5, thus we treat this value as a long term minimum assumption in our estimates. Our estimate for people per household is used to calculate the total number of homes within a country. Our model is relatively insensitive to this input because the proportion of homes exposed to lead paint has a greater effect on our final estimate of cost-effectiveness than the absolute number of homes within the country.

In [396]:
PEOPLE_PER_HOUSEHOLD_INIT = pd.Series([3.96, 4.18, 9.01, 4.33, 6.45, 8.21, 7.45, 3.27, 4.86, 4.22, 6.86, 2.81, 10.8],
                                      index=COUNTRIES)
PEOPLE_PER_HOUSEHOLD_MIN = 2.5
PEOPLE_PER_HOUSEHOLD_RATE = -0.01

people_per_household = pd.DataFrame(
    np.array(
        [np.maximum(
            PEOPLE_PER_HOUSEHOLD_INIT * (1 + PEOPLE_PER_HOUSEHOLD_RATE)**(year - YEARS[0]), PEOPLE_PER_HOUSEHOLD_MIN)
         for year in YEARS]),
    index=YEARS, columns=COUNTRIES)
people_per_household.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
2021,3.960,4.180,9.010,4.330,6.450,8.210,7.450,3.270,4.860,4.220,6.860,2.810,10.800
2022,3.920,4.138,8.920,4.287,6.386,8.128,7.375,3.237,4.811,4.178,6.791,2.782,10.692
2023,3.881,4.097,8.831,4.244,6.322,8.047,7.302,3.205,4.763,4.136,6.723,2.754,10.585
2024,3.842,4.056,8.742,4.201,6.258,7.966,7.229,3.173,4.716,4.095,6.656,2.727,10.479
2025,3.804,4.015,8.655,4.159,6.196,7.886,7.156,3.141,4.668,4.054,6.590,2.699,10.374


#### Number of homes

In [397]:
number_of_homes = (population / people_per_household)
number_of_homes.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
Year,,,,,,,,,,,,,
2021,"5,022,662.121","6,917,620.096","934,588.457","1,199,404.619","5,349,422.636","28,185,397.077","3,389,627.114","4,890,986.239","2,485,488.477","7,780,340.284","31,108,065.598","9,778,736.655","1,562,659.815"
2022,"5,204,907.152","7,155,700.063","964,777.968","1,237,009.821","5,573,407.877","29,014,242.301","3,553,383.771","5,041,404.566","2,540,656.358","8,012,798.602","32,179,111.818","10,122,775.082","1,619,570.894"
2023,"5,393,118.513","7,402,265.856","995,514.626","1,276,765.132","5,802,953.503","29,886,539.953","3,725,526.432","5,199,933.415","2,600,846.349","8,249,953.699","33,286,994.276","10,483,729.418","1,678,132.239"
2024,"5,589,228.400","7,657,238.157","1,026,947.425","1,317,884.758","6,040,594.647","30,781,451.838","3,906,493.336","5,364,316.387","2,665,027.225","8,493,381.351","34,426,618.705","10,857,461.081","1,738,826.363"
2025,"5,792,382.371","7,919,054.298","1,058,973.162","1,360,114.973","6,285,309.045","31,693,285.257","4,096,446.400","5,531,506.548","2,730,247.458","8,742,545.685","35,597,068.878","11,241,606.869","1,801,332.079"


#### Growth in the paint market

We use data from a [variety of sources](https://docs.google.com/spreadsheets/d/1bIiee05WlPt4B7NV3uvol2vu5JVbwOvL/edit#gid=508218074)  – IndustryArc, European Coatings, and Emerald Group, among others – to calculate an initial paint market growth rate of 5.5%. We arrive at this estimate by considering the average of the paint market growth rates from these sources, which is 5.75%. However, these growth rates correspond to a range of different countries and regions, and reflect growth in the paint market as a whole, rather than in the portion of the market that is specifically captured by home-use paints. Therefore, to be more conservative, we choose 5.5% as the initial value in 2021. 5.5% is also in line with a 20-year average of the World Bank’s estimate for real [GDP growth](https://data.worldbank.org/indicator/NY.GDP.MKTP.KD.ZG?locations=XO) in low- and middle-income countries (LMICs), which we would expect to approximate future GDP growth and correlate with paint market growth.

We assume that the paint market growth rate decreases by 0.03% per year (i.e. 5.5% in year 1, 5.47% in year 2). We expect that once all existing homes within a country are painted, then growth in the paint market will be roughly equal to population growth from that point forward. To reflect our expectation that growth in the paint market will converge with population growth in the long term, we treat the population growth rate in each country as the long term minimum for paint market growth.

This figure is a simplification in that we use one estimate of paint market growth rate for all countries within our model, rather than calculating unique growth rates for each country.


In [398]:
# TODO(drw): Modeling population(/housing) and paint growth independently seems like a mistake
GROWTH_IN_THE_PAINT_MARKET_INIT = pd.Series(sq.norm(0.025, 0.08) @ N)
GROWTH_IN_THE_PAINT_MARKET_RATE = -0.0003

growth_in_the_paint_market = pd.DataFrame(
    np.array(
        [GROWTH_IN_THE_PAINT_MARKET_INIT + GROWTH_IN_THE_PAINT_MARKET_RATE*(year - YEARS[0])
         for year in YEARS]).T,
    index=SAMPLES, columns=YEARS)

growth_in_the_paint_market.head()

,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,0.081,0.081,0.081,0.080,0.080,0.080,0.080,0.079,0.079,0.079,...,0.060,0.060,0.060,0.059,0.059,0.059,0.059,0.058,0.058,0.058
1,0.027,0.027,0.026,0.026,0.026,0.025,0.025,0.025,0.024,0.024,...,0.006,0.006,0.005,0.005,0.005,0.004,0.004,0.004,0.003,0.003
2,0.067,0.067,0.066,0.066,0.066,0.065,0.065,0.065,0.065,0.064,...,0.046,0.046,0.045,0.045,0.045,0.044,0.044,0.044,0.044,0.043
3,0.047,0.047,0.046,0.046,0.046,0.046,0.045,0.045,0.045,0.044,...,0.026,0.026,0.025,0.025,0.025,0.025,0.024,0.024,0.024,0.023
4,0.051,0.051,0.051,0.051,0.050,0.050,0.050,0.049,0.049,0.049,...,0.030,0.030,0.030,0.030,0.029,0.029,0.029,0.028,0.028,0.028


#### % population urban

We use UN population projections of urban populations to determine the proportion of each country’s population that lives in an urban area. We use the UN’s estimate for the urban percentage of the population in 2020.

In our Malawi CEA, we assumed that the proportion of homes that are painted is equal to the proportion of the population that lives in urban areas. In our current CEA, we update this starting assumption such that in Year 1, the proportion of homes that are painted is equal to the sum of 75% of the urban population and 25% of the rural (non-urban) population. In our model, the urban population percentage is only used to calculate the percentage of homes painted in Year 1 (2021). In future years, our estimate for the percentage of homes painted is calculated using a different formula that is primarily dependent on changes in the paint market growth rate, rather than urban population projections. We use this approach because we are uncertain about how urban-rural classification will correspond with home painting rates in future years. For example, it is possible that the proportion of homes painted in rural areas will increase significantly as countries continue to experience economic growth.


In [399]:
pct_population_urban = pd.DataFrame(
    np.array(
        [sq.lognorm(0.165, 0.183) @ N,
         sq.lognorm(0.371, 0.410) @ N,
         sq.lognorm(0.409, 0.452) @ N,
         sq.lognorm(0.494, 0.546) @ N,
         sq.lognorm(0.637, 0.704) @ N,
         sq.lognorm(0.353, 0.391) @ N,
         sq.lognorm(0.162, 0.179) @ N,
         sq.lognorm(0.304, 0.336) @ N,
         sq.lognorm(0.665, 0.735) @ N,
         sq.lognorm(0.542, 0.599) @ N,
         sq.lognorm(0.494, 0.546) @ N,
         sq.lognorm(0.494, 0.546) @ N,
         sq.lognorm(0.456, 0.504) @ N]).T,
    columns=COUNTRIES)
pct_population_urban.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
0,0.180,0.399,0.450,0.484,0.677,0.370,0.170,0.321,0.652,0.567,0.496,0.521,0.464
1,0.176,0.388,0.438,0.524,0.687,0.376,0.177,0.333,0.700,0.573,0.484,0.506,0.486
2,0.184,0.404,0.421,0.520,0.658,0.383,0.175,0.304,0.689,0.584,0.509,0.510,0.477
3,0.176,0.399,0.417,0.526,0.686,0.359,0.175,0.308,0.709,0.563,0.531,0.496,0.453
4,0.178,0.412,0.430,0.546,0.692,0.370,0.173,0.304,0.714,0.579,0.515,0.506,0.493


#### Number of painted homes

In [400]:
PCT_OF_HOMES_PAINTED_URBAN = 0.75
PCT_OF_HOMES_PAINTED_RURAL = 0.25
PCT_OF_HOMES_PAINTED_INIT = PCT_OF_HOMES_PAINTED_URBAN*pct_population_urban + \
    PCT_OF_HOMES_PAINTED_RURAL*(1 - pct_population_urban)

number_of_painted_homes = pd.DataFrame(
    index=SAMPLE_YEAR_INDEX, columns=COUNTRIES, dtype=np.float64)
number_of_painted_homes.loc[idx[:, YEARS[0]], :] = \
    (number_of_homes.loc[YEARS[0]] * PCT_OF_HOMES_PAINTED_INIT).values

for year in YEARS[1:]:
    number_of_painted_homes.loc[idx[:, year], :] = np.minimum(
        number_of_painted_homes.loc[idx[:, year - 1], :].mul(
            1 + growth_in_the_paint_market[year], level=0, axis=0).values,
        number_of_homes.loc[year].values)

number_of_painted_homes.head()
# number_of_painted_homes.loc[idx[:, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi    Madagascar  Sierra Leone     Liberia  \
Sample Year                                                         
0      2021 1,708,527.214 3,110,836.176   444,008.182 590,334.709   
       2022 1,847,053.311 3,363,060.424   480,008.030 638,198.602   
       2023 1,996,256.883 3,634,725.906   518,782.716 689,751.803   
       2024 2,156,914.130 3,927,245.903   560,533.957 745,262.507   
       2025 2,329,853.876 4,242,129.514   605,477.147 805,017.093   

                   Angola       Pakistan         Niger      Zimbabwe  \
Sample Year                                                            
0      2021 3,148,680.263 12,257,231.479 1,136,113.222 2,006,894.959   
       2022 3,403,972.881 13,251,038.553 1,228,228.424 2,169,612.487   
       2023 3,678,943.241 14,321,447.443 1,327,443.789 2,344,872.146   
       2024 3,975,021.815 15,474,026.720 1,434,275.463 2,533,585.686   
       2025 4,293,736.063 16,714,722.500 1,549,274.587 2,736,726.673   

                  Bolivia         Ghana        Nigeria  Cote d'Ivoire  \
Sample Year                                                             
0      2021 1,431,239.118 4,149,587.565 15,498,828.597  4,991,102.697   
       2022 1,547,282.905 4,486,032.991 16,755,461.918  5,395,777.535   
       2023 1,672,271.250 4,848,411.350 18,108,955.481  5,831,644.370   
       2024 1,806,854.378 5,238,607.834 19,566,350.546  6,300,970.706   
       2025 1,951,726.598 5,658,635.455 21,135,165.761  6,806,177.781   

                  Senegal  
Sample Year                
0      2021   753,562.317  
       2022   814,660.581  
       2023   880,468.248  
       2024   951,327.667  
       2025 1,027,604.402

#### % of homes painted

In [401]:
pct_of_homes_painted = number_of_painted_homes / number_of_homes
pct_of_homes_painted.head()
# pct_of_homes_painted.loc[idx[:, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi  Madagascar  Sierra Leone  Liberia  Angola  Pakistan  \
Sample Year                                                                
0      2021   0.340       0.450         0.475    0.492   0.589     0.435   
       2022   0.355       0.470         0.498    0.516   0.611     0.457   
       2023   0.370       0.491         0.521    0.540   0.634     0.479   
       2024   0.386       0.513         0.546    0.565   0.658     0.503   
       2025   0.402       0.536         0.572    0.592   0.683     0.527   

             Niger  Zimbabwe  Bolivia  Ghana  Nigeria  Cote d'Ivoire  Senegal  
Sample Year                                                                    
0      2021  0.335     0.410    0.576  0.533    0.498          0.510    0.482  
       2022  0.346     0.430    0.609  0.560    0.521          0.533    0.503  
       2023  0.356     0.451    0.643  0.588    0.544          0.556    0.525  
       2024  0.367     0.472    0.678  0.617    0.568          0.580    0.547  
       2025  0.378     0.495    0.715  0.647    0.594          0.605    0.570

#### New painted homes

In [402]:
# TODO(drw): I'm a little skeptical that new paint will grow exponentially and suddenly hit a ceiling
new_painted_homes = number_of_painted_homes.groupby("Sample").diff()
new_painted_homes.head()
# new_painted_homes.loc[idx[:, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi  Madagascar  Sierra Leone    Liberia      Angola  \
Sample Year                                                                
0      2021         NaN         NaN           NaN        NaN         NaN   
       2022 138,526.097 252,224.248    35,999.848 47,863.893 255,292.618   
       2023 149,203.572 271,665.482    38,774.686 51,553.201 274,970.359   
       2024 160,657.247 292,519.997    41,751.241 55,510.704 296,078.574   
       2025 172,939.746 314,883.611    44,943.189 59,754.585 318,714.248   

                 Pakistan       Niger    Zimbabwe     Bolivia       Ghana  \
Sample Year                                                                 
0      2021           NaN         NaN         NaN         NaN         NaN   
       2022   993,807.074  92,115.202 162,717.528 116,043.787 336,445.427   
       2023 1,070,408.890  99,215.365 175,259.659 124,988.345 362,378.358   
       2024 1,152,579.278 106,831.674 188,713.540 134,583.128 390,196.485   
       2025 1,240,695.780 114,999.124 203,140.987 144,872.220 420,027.621   

                  Nigeria  Cote d'Ivoire    Senegal  
Sample Year                                          
0      2021           NaN            NaN        NaN  
       2022 1,256,633.321    404,674.838 61,098.264  
       2023 1,353,493.563    435,866.835 65,807.667  
       2024 1,457,395.065    469,326.336 70,859.420  
       2025 1,568,815.214    505,207.074 76,276.734

#### % of prior year painted homes repainted

In our [first cost-effectiveness analysis](https://my.causal.app/models/61262) evaluating LEEP’s Malawi program, we used a conservative estimate for the repainting rate (2%) due to the relative lack of information available to inform this approximation. Since the original Malawi CEA was published, we have conducted informal surveys within our program countries to gather more information about repainting rates from relevant stakeholders. Data from these surveys suggest that repainting actually occurs about every 5-20 years within focus countries – implying a repainting rate within the range of 5-20%. In this CEA, we estimate the repainting rate at 4%. This is still a conservative estimate. In the future, we hope to conduct more formal data collection procedures that will allow us to further hone this estimate.

In [403]:
pct_of_prior_year_painted_homes_repainted = pd.Series(
    sq.lognorm(0.02, 0.1) @ N)
pct_of_prior_year_painted_homes_repainted.head()

0   0.041
1   0.053
2   0.053
3   0.085
4   0.038
dtype: float64

#### Number of homes repainted

In [404]:
number_of_homes_repainted = pd.DataFrame(
    index=SAMPLE_YEAR_INDEX, columns=COUNTRIES, dtype=np.float64)

for year in YEARS[1:]:
    number_of_homes_repainted.loc[idx[:, year], :] = \
        number_of_painted_homes.loc[idx[:, year], :].mul(
        pct_of_prior_year_painted_homes_repainted, level=0, axis=0).values

number_of_homes_repainted.head()
# number_of_homes_repainted.loc[idx[:, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi  Madagascar  Sierra Leone    Liberia      Angola  \
Sample Year                                                               
0      2021        NaN         NaN           NaN        NaN         NaN   
       2022 75,298.330 137,100.988    19,568.359 26,017.272 138,768.855   
       2023 81,380.872 148,175.902    21,149.077 28,118.928 149,978.498   
       2024 87,930.343 160,100.987    22,851.138 30,381.918 162,048.655   
       2025 94,980.532 172,937.763    24,683.325 32,817.917 175,041.593   

               Pakistan      Niger    Zimbabwe    Bolivia       Ghana  \
Sample Year                                                             
0      2021         NaN        NaN         NaN        NaN         NaN   
       2022 540,201.556 50,070.861  88,448.014 63,077.670 182,880.911   
       2023 583,838.630 54,115.547  95,592.778 68,173.036 197,653.893   
       2024 630,825.522 58,470.725 103,286.012 73,659.551 213,560.929   
       2025 681,404.636 63,158.864 111,567.407 79,565.518 230,684.083   

                Nigeria  Cote d'Ivoire    Senegal  
Sample Year                                        
0      2021         NaN            NaN        NaN  
       2022 683,065.449    219,968.224 33,211.051  
       2023 738,242.960    237,737.091 35,893.814  
       2024 797,656.196    256,869.993 38,782.521  
       2025 861,611.668    277,465.635 41,892.075

### Scenarios

#### Start of enforcement/compliance

The predicted start dates for enforcement/compliance in the counterfactual v.s. LEEP scenarios reflect a best guess estimate for the number of years that compliance to regulation is brought forward due to LEEP’s activities.

Our estimate for the year in which enforcement/compliance begins in the counterfactual scenario is based on our understanding of how much progress a country has made toward implementing lead paint regulation, as well as any barriers or catalysts identified, independent of LEEP intervention. We expect that enforcement of lead paint regulation will be initiated more quickly in countries where governments are already aware of the threats of lead poisoning and/or have already drafted regulation that bans lead paint manufacturing. For example, in Pakistan, where implementation has already begun, the start date for enforcement/compliance under the LEEP scenario is 2023 versus 2028 in the counterfactual scenario. In Angola, the start date is 2025 under the LEEP scenario and 2032 under the counterfactual scenario.

Historically, LMIC governments have very rarely implemented lead paint regulation without externally funded paint studies and advocacy. Of 38 LMICs with lead paint regulation, we were only able to identify one in Africa (Algeria), five in Latin America (all upper-middle income countries), and five in Europe (as a result of EU laws) that implemented lead paint regulation without an externally )funded paint study and advocacy. At present, support for progress toward lead paint regulation in LMICs is very limited, in part because the SAICM’s [Global Environment Facility Project](https://www.saicm.org/Implementation/GEFProject/tabid/7893/language/en-US/Default.aspx) has come to an end.
Under the LEEP scenario, we assume countries that have made more progress toward implementing lead paint regulation prior to intervention will achieve compliance objectives sooner with LEEP’s support than those that have made less regulatory progress. Our estimate for the LEEP scenario is also informed by observed progress within a given country since LEEP started working there. 


In [405]:
start_of_enforcement_compliance = pd.Series([2031, 2024], index=SCENARIOS)
start_of_enforcement_compliance.head()

Counterfactual       2031
LEEP intervention    2024
dtype: int64

#### Lead market share of colored solvent based paint

For each country, we estimate the proportion of paint within the coloured solvent-based paint market that has a lead concentration greater than 90 parts per million (ppm). Estimates are primarily based on studies conducted by LEEP and by [IPEN](https://ipen.org/) that evaluate lead content among paints produced and sold within a given country. We use domestic and regional industry data to inform our estimates for countries in which a formal paint study has not been conducted. We adjust estimates slightly to reflect any information available about the relative market share of different brands within a given country. It is possible that we overestimate the value of this input because the paint studies that inform this estimate primarily test yellow and red paints, which are more likely to contain lead than other coloured paints, such as blue paint.

At the same time, because we only consider the contribution of the coloured solvent-based paint market to overall lead paint exposure, our approach is also at risk of underestimating the overall number of homes that contain lead paint. For simplicity, we ignore the contribution of both white paints and water-based paints, which on average are less likely to contain high levels of lead. However, evidence indicates that these types of paint may still, in some cases, contain high levels of lead. Across studies of solvent-based paints in the countries included in this CEA, 26% of white paints had lead content >90 ppm. In a study of water-based paints [in Nigeria](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6221508/), all tested samples had lead content >90 ppm. LEEP typically does not test the lead content of water-based paints, but has also identified high lead levels in some countries. 

In [406]:
lead_market_share_of_colored_solvent_based_paint = pd.DataFrame(
    index=SCENARIO_SAMPLE_YEAR_INDEX, columns=COUNTRIES, dtype=np.float64)

for scenario in SCENARIOS:
    lead_market_share_of_colored_solvent_based_paint.loc[idx[scenario, :, YEARS[0]], :] = \
        lead_market_share_of_colored_solvent_based_paints_2021.values

    for year in YEARS[1:]:
        if year < start_of_enforcement_compliance[scenario]:
            lead_market_share_of_colored_solvent_based_paint.loc[idx[scenario, :, year], :] = \
                lead_market_share_of_colored_solvent_based_paint.loc[idx[scenario,
                                                                         :, year - 1], :].values
        else:
            lead_market_share_of_colored_solvent_based_paint.loc[idx[scenario, :, year], :] = \
                lead_market_share_of_colored_solvent_based_paint.loc[idx[scenario, :, year - 1], :].mul(
                (1 + decline_in_lead_market_share_once_enforcement_compliance_begins), level=1, axis=0).values

lead_market_share_of_colored_solvent_based_paint.head()
# lead_market_share_of_colored_solvent_based_paint.loc[idx['Counterfactual', :, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi  Madagascar  Sierra Leone  Liberia  Angola  \
Scenario       Sample Year                                                      
Counterfactual 0      2021   0.693       0.872         0.403    0.726   0.478   
                      2022   0.693       0.872         0.403    0.726   0.478   
                      2023   0.693       0.872         0.403    0.726   0.478   
                      2024   0.693       0.872         0.403    0.726   0.478   
                      2025   0.693       0.872         0.403    0.726   0.478   

                            Pakistan  Niger  Zimbabwe  Bolivia  Ghana  \
Scenario       Sample Year                                              
Counterfactual 0      2021     0.539  0.392     0.654    0.362  0.196   
                      2022     0.539  0.392     0.654    0.362  0.196   
                      2023     0.539  0.392     0.654    0.362  0.196   
                      2024     0.539  0.392     0.654    0.362  0.196   
                      2025     0.539  0.392     0.654    0.362  0.196   

                            Nigeria  Cote d'Ivoire  Senegal  
Scenario       Sample Year                                   
Counterfactual 0      2021    0.723          0.679    0.735  
                      2022    0.723          0.679    0.735  
                      2023    0.723          0.679    0.735  
                      2024    0.723          0.679    0.735  
                      2025    0.723          0.679    0.735

#### Number of homes with lead paint

In [407]:
# TODO(drw): Verify that % of new painted homes is used here as an approx. of lead paint in existing homes
PCT_OF_PAINTED_HOMES_WITH_LEAD_PAINT_INIT = pd.DataFrame(
    index=SCENARIO_SAMPLE_INDEX, columns=COUNTRIES, dtype=np.float64)
for scenario in SCENARIOS:
    PCT_OF_PAINTED_HOMES_WITH_LEAD_PAINT_INIT.loc[idx[scenario, :], :] = \
        lead_market_share_of_colored_solvent_based_paint.loc[idx[scenario, :, YEARS[0]], :].mul(
        pct_of_new_painted_homes_that_use_some_colored_solvent_based_lead_paint, level=1, axis=0).values

number_of_homes_with_lead_paint = pd.DataFrame(
    index=SCENARIO_SAMPLE_YEAR_INDEX, columns=COUNTRIES, dtype=np.float64)

for scenario in SCENARIOS:
    number_of_homes_with_lead_paint.loc[idx[scenario, :, YEARS[0]], :] = \
        PCT_OF_PAINTED_HOMES_WITH_LEAD_PAINT_INIT.loc[idx[scenario, :], :] * \
        number_of_painted_homes.loc[idx[:, YEARS[0]], :]

    for year in YEARS[1:]:
        number_of_homes_with_lead_paint.loc[idx[scenario, :, year], :] = \
            number_of_homes_with_lead_paint.loc[idx[scenario, :, year - 1], :].mul(
            (1 - pct_of_prior_year_painted_homes_repainted), level=1, axis=0).values + \
            (lead_market_share_of_colored_solvent_based_paint.loc[idx[scenario, :, year], :] *
             (number_of_homes_repainted.loc[idx[:, year], :] + new_painted_homes.loc[idx[:, year], :])).mul(
            pct_of_new_painted_homes_that_use_some_colored_solvent_based_lead_paint, level=1, axis=0).values

number_of_homes_with_lead_paint.head()
# number_of_homes_with_lead_paint.loc[idx['LEEP intervention', :, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi    Madagascar  Sierra Leone  \
Scenario       Sample Year                                             
Counterfactual 0      2021   987,813.747 2,262,533.458   149,381.644   
                      2022 1,071,169.996 2,453,456.395   161,987.151   
                      2023 1,160,818.169 2,658,790.642   175,544.152   
                      2024 1,257,220.500 2,879,594.919   190,122.547   
                      2025 1,360,870.343 3,116,999.225   205,796.943   

                               Liberia        Angola      Pakistan  \
Scenario       Sample Year                                           
Counterfactual 0      2021 357,648.374 1,256,404.031 5,515,860.734   
                      2022 387,828.382 1,362,425.159 5,981,314.331   
                      2023 420,286.447 1,476,449.008 6,481,901.450   
                      2024 455,189.927 1,599,063.498 7,020,203.167   
                      2025 492,717.445 1,730,896.123 7,598,974.315   

                                 Niger      Zimbabwe     Bolivia       Ghana  \
Scenario       Sample Year                                                     
Counterfactual 0      2021 371,778.273 1,094,298.842 431,989.250 677,481.819   
                      2022 403,150.627 1,186,640.792 468,442.482 734,650.838   
                      2023 436,891.039 1,285,952.927 507,647.289 796,135.109   
                      2024 473,173.478 1,392,747.310 549,805.814 862,251.649   
                      2025 512,183.625 1,507,570.477 595,133.811 933,338.819   

                                  Nigeria  Cote d'Ivoire     Senegal  
Scenario       Sample Year                                            
Counterfactual 0      2021  9,348,146.125  2,827,938.660 462,215.360  
                      2022 10,136,985.518  3,066,573.079 501,219.209  
                      2023 10,985,368.349  3,323,220.180 543,167.160  
                      2024 11,897,668.958  3,599,203.260 588,275.500  
                      2025 12,878,556.171  3,895,934.701 636,775.077

#### % of total homes with lead paint

In [408]:
pct_of_total_homes_with_lead_paint = number_of_homes_with_lead_paint/number_of_homes

pct_of_total_homes_with_lead_paint.head()
# pct_of_total_homes_with_lead_paint.loc[idx['Counterfactual', :, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi  Madagascar  Sierra Leone  Liberia  Angola  \
Scenario       Sample Year                                                      
Counterfactual 0      2021   0.197       0.327         0.160    0.298   0.235   
                      2022   0.206       0.343         0.168    0.314   0.244   
                      2023   0.215       0.359         0.176    0.329   0.254   
                      2024   0.225       0.376         0.185    0.345   0.265   
                      2025   0.235       0.394         0.194    0.362   0.275   

                            Pakistan  Niger  Zimbabwe  Bolivia  Ghana  \
Scenario       Sample Year                                              
Counterfactual 0      2021     0.196  0.110     0.224    0.174  0.087   
                      2022     0.206  0.113     0.235    0.184  0.092   
                      2023     0.217  0.117     0.247    0.195  0.097   
                      2024     0.228  0.121     0.260    0.206  0.102   
                      2025     0.240  0.125     0.273    0.218  0.107   

                            Nigeria  Cote d'Ivoire  Senegal  
Scenario       Sample Year                                   
Counterfactual 0      2021    0.301          0.289    0.296  
                      2022    0.315          0.303    0.309  
                      2023    0.330          0.317    0.324  
                      2024    0.346          0.331    0.338  
                      2025    0.362          0.347    0.354

#### Births in homes with lead paint

In [409]:
births_in_homes_with_lead_paint_counterfactual = pct_of_total_homes_with_lead_paint*births

births_in_homes_with_lead_paint_counterfactual.head()
# births_in_homes_with_lead_paint_counterfactual.loc[idx[:, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi  Madagascar  Sierra Leone    Liberia  \
Scenario       Sample Year                                                    
Counterfactual 0      2021 128,564.652 292,740.745    42,213.862 48,510.757   
                      2022 136,996.546 310,682.452    44,497.617 51,559.132   
                      2023 146,140.201 329,787.391    47,041.615 54,966.265   
                      2024 154,744.945 348,680.759    49,471.234 58,424.488   
                      2025 164,060.980 369,086.153    52,105.647 62,091.278   

                                Angola      Pakistan       Niger    Zimbabwe  \
Scenario       Sample Year                                                     
Counterfactual 0      2021 314,438.357 1,247,531.957 125,515.952 109,364.637   
                      2022 332,385.654 1,324,508.468 133,951.588 115,616.990   
                      2023 351,557.462 1,404,184.906 142,708.557 122,127.524   
                      2024 371,715.222 1,485,206.053 152,132.952 128,947.449   
                      2025 392,945.598 1,572,835.900 162,061.320 135,782.568   

                              Bolivia      Ghana       Nigeria  Cote d'Ivoire  \
Scenario       Sample Year                                                      
Counterfactual 0      2021 45,853.121 78,775.934 2,380,993.761    269,800.248   
                      2022 48,688.838 83,128.386 2,521,009.356    285,710.810   
                      2023 51,516.459 87,808.212 2,676,175.722    304,296.779   
                      2024 54,719.045 92,515.458 2,845,767.521    323,207.788   
                      2025 58,023.775 97,889.195 3,010,448.303    343,091.639   

                               Senegal  
Scenario       Sample Year              
Counterfactual 0      2021 162,678.435  
                      2022 171,974.880  
                      2023 182,060.593  
                      2024 192,289.216  
                      2025 203,553.709

### Births with averted lead paint exposure vs. counterfactual

In [410]:
births_with_averted_lead_paint_exposure_vs_counterfactual = \
    births_in_homes_with_lead_paint_counterfactual.loc[idx['Counterfactual', :, :], :].droplevel(0) - \
    births_in_homes_with_lead_paint_counterfactual.loc[idx['LEEP intervention', :, :], :].droplevel(
        0)

births_with_averted_lead_paint_exposure_vs_counterfactual.head()
# births_with_averted_lead_paint_exposure_vs_counterfactual.loc[idx[:, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi  Madagascar  Sierra Leone   Liberia     Angola  \
Sample Year                                                            
0      2021     0.000       0.000         0.000     0.000      0.000   
       2022     0.000       0.000         0.000     0.000      0.000   
       2023     0.000       0.000         0.000     0.000      0.000   
       2024 3,210.357   7,233.772     1,026.336 1,212.081  7,711.647   
       2025 9,179.067  20,650.044     2,915.265 3,473.952 21,984.959   

              Pakistan     Niger  Zimbabwe   Bolivia     Ghana     Nigeria  \
Sample Year                                                                  
0      2021      0.000     0.000     0.000     0.000     0.000       0.000   
       2022      0.000     0.000     0.000     0.000     0.000       0.000   
       2023      0.000     0.000     0.000     0.000     0.000       0.000   
       2024 30,812.257 3,156.168 2,675.159 1,135.208 1,919.336  59,038.623   
       2025 87,998.779 9,067.188 7,596.915 3,246.379 5,476.814 168,431.922   

             Cote d'Ivoire    Senegal  
Sample Year                            
0      2021          0.000      0.000  
       2022          0.000      0.000  
       2023          0.000      0.000  
       2024      6,705.306  3,989.254  
       2025     19,195.674 11,388.650

### Total number of births with averted lead paint exposure

In [411]:
total_number_of_births_with_averted_lead_paint_exposure = \
    births_with_averted_lead_paint_exposure_vs_counterfactual.groupby(
        level=0).sum()

total_number_of_births_with_averted_lead_paint_exposure.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
Sample,,,,,,,,,,,,,
0,"2,877,654.711","6,323,169.142","800,438.630","1,019,867.336","6,181,548.420","26,347,777.400","2,971,456.790","2,478,589.588","765,881.108","1,561,965.779","47,323,529.552","6,600,764.820","3,580,547.889"
1,"881,416.721","1,647,455.449","278,545.770","390,718.113","2,586,887.612","9,357,871.076","860,031.949","774,584.867","414,243.405","585,226.645","18,807,918.923","2,070,500.981","1,085,573.954"
2,"2,318,823.362","3,533,637.014","503,014.142","653,905.362","5,524,657.457","20,142,240.096","1,910,367.462","1,552,563.306","625,812.813","1,092,048.036","39,989,531.613","5,329,842.068","2,053,099.358"
3,"1,412,769.211","2,443,589.488","402,490.907","543,352.018","4,320,844.762","16,470,740.983","1,348,114.940","1,141,790.252","590,631.792","750,835.385","33,802,281.059","3,494,503.003","1,817,619.211"
4,"1,467,223.752","3,037,890.982","426,700.755","579,139.491","4,626,606.419","17,029,970.934","1,305,037.717","1,049,479.671","546,963.128","942,215.584","33,249,339.619","3,411,501.664","1,801,127.975"


### Total expected number of births with averted lead paint exposure

In [412]:
total_expected_number_of_births_with_averted_lead_paint_exposure = \
    total_number_of_births_with_averted_lead_paint_exposure * \
    probability_of_success

total_expected_number_of_births_with_averted_lead_paint_exposure.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
Sample,,,,,,,,,,,,,
0,"2,529,270.739","5,305,846.563","642,402.689","722,617.248","5,110,593.485","23,497,854.067","2,095,392.755","1,914,061.365","564,629.763","1,235,879.510","36,761,609.568","5,036,773.078","2,652,171.277"
1,"804,085.976","1,326,412.948","206,931.193","290,028.363","1,963,816.620","8,478,305.161","628,341.056","616,242.363","314,319.019","462,554.081","15,198,840.212","1,443,495.281","817,415.458"
2,"2,051,909.236","2,777,615.013","370,611.821","458,367.170","4,416,229.063","18,208,399.063","1,448,019.818","1,159,432.550","502,357.008","879,666.613","31,811,114.978","4,141,498.589","1,630,962.151"
3,"1,217,996.967","2,034,923.871","319,604.485","390,151.825","3,386,948.220","14,267,954.118","1,059,444.192","862,155.536","424,633.468","586,960.459","29,226,222.550","2,663,729.312","1,225,606.603"
4,"1,262,707.953","2,465,453.752","305,282.628","369,861.744","3,648,281.373","15,350,194.622","948,122.751","908,650.478","405,254.861","765,917.285","28,185,162.075","2,440,420.113","1,334,944.830"


### Number of homes with averted lead paint vs. counterfactual

In [413]:
number_of_homes_with_averted_lead_paint_vs_counterfactual = \
    number_of_homes_with_lead_paint.loc[idx['Counterfactual', :, :], :].droplevel(0) - \
    number_of_homes_with_lead_paint.loc[idx['LEEP intervention', :, :], :].droplevel(
        0)

number_of_homes_with_averted_lead_paint_vs_counterfactual.head()

Malawi  Madagascar  Sierra Leone    Liberia     Angola  \
Sample Year                                                              
0      2021      0.000       0.000         0.000      0.000      0.000   
       2022      0.000       0.000         0.000      0.000      0.000   
       2023      0.000       0.000         0.000      0.000      0.000   
       2024 26,082.442  59,740.410     3,944.304  9,443.423 33,174.357   
       2025 76,139.493 174,393.352    11,514.157 27,567.106 96,842.109   

               Pakistan      Niger   Zimbabwe    Bolivia      Ghana  \
Sample Year                                                           
0      2021       0.000      0.000      0.000      0.000      0.000   
       2022       0.000      0.000      0.000      0.000      0.000   
       2023       0.000      0.000      0.000      0.000      0.000   
       2024 145,641.950  9,816.512 28,894.097 11,406.335 17,888.373   
       2025 425,155.898 28,656.221 84,347.236 33,297.211 52,219.482   

                Nigeria  Cote d'Ivoire    Senegal  
Sample Year                                        
0      2021       0.000          0.000      0.000  
       2022       0.000          0.000      0.000  
       2023       0.000          0.000      0.000  
       2024 246,830.421     74,669.489 12,204.432  
       2025 720,543.838    217,974.104 35,627.003

### Total number of homes with averted lead paint

In [414]:
total_number_of_homes_with_averted_lead_paint = \
    number_of_homes_with_averted_lead_paint_vs_counterfactual.groupby(
        level=0).sum()

total_number_of_homes_with_averted_lead_paint.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
Sample,,,,,,,,,,,,,
0,"42,267,381.556","90,452,253.272","5,668,410.291","13,580,644.629","46,945,672.870","215,797,918.614","16,391,894.543","43,975,021.342","12,914,427.360","23,700,080.330","357,923,436.978","103,873,322.950","18,410,612.094"
1,"11,458,275.885","21,287,851.413","1,773,403.265","4,717,107.957","17,692,562.743","69,050,705.487","4,131,206.449","12,616,520.229","6,490,710.259","8,196,393.188","127,326,024.905","30,996,905.087","5,044,347.426"
2,"33,170,425.540","49,453,834.525","3,481,519.001","8,502,022.426","41,064,526.834","160,851,631.814","10,163,361.911","27,064,100.121","10,275,370.017","16,206,417.238","294,781,696.510","82,716,405.394","10,315,118.965"
3,"18,331,371.805","31,449,356.637","2,551,913.014","6,529,597.220","29,362,302.501","120,873,464.377","6,422,634.775","18,593,096.809","9,123,295.010","10,445,173.368","227,620,090.029","52,051,902.958","8,383,732.680"
4,"20,310,573.342","41,697,758.860","2,907,751.571","7,444,372.392","33,873,397.980","134,101,019.305","6,720,043.338","17,952,018.583","9,092,184.273","13,974,118.494","241,469,794.267","52,884,439.610","8,906,890.949"


### Total expected number of homes with averted lead paint

In [415]:
total_expected_number_of_homes_with_averted_lead_paint = \
    total_number_of_homes_with_averted_lead_paint * \
    probability_of_success

total_expected_number_of_homes_with_averted_lead_paint.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
Sample,,,,,,,,,,,,,
0,"37,150,270.673","75,899,563.401","4,549,258.216","9,622,435.881","38,812,322.355","192,456,005.778","11,559,130.583","33,959,187.839","9,520,890.365","18,752,295.385","278,040,158.250","79,261,475.121","13,637,046.089"
1,"10,452,988.607","17,139,450.885","1,317,458.363","3,501,488.799","13,431,178.302","62,560,484.956","3,018,267.668","10,037,420.772","4,925,011.855","6,478,302.297","102,893,250.192","21,610,173.890","3,798,292.640"
2,"29,352,258.415","38,873,181.559","2,565,120.918","5,959,651.330","32,825,629.155","145,408,389.931","7,703,622.342","20,211,091.225","8,248,319.676","13,054,594.383","234,494,730.566","64,273,926.304","8,194,230.132"
3,"15,804,106.634","26,189,769.947","2,026,388.252","4,688,552.150","23,016,008.134","104,707,920.888","5,047,361.254","14,039,479.942","6,559,173.504","8,165,443.289","196,805,517.250","39,677,224.344","5,653,086.232"
4,"17,479,489.720","33,840,548.145","2,080,347.949","4,754,275.264","26,710,654.788","120,873,767.390","4,882,177.654","15,543,045.479","6,736,563.550","11,359,416.129","204,691,743.228","37,830,921.034","6,601,534.253"


## Benefits

### Income benefit

#### IQ loss per BLL increase

We assume that a 1 ug/dL increase in blood lead level is associated with a 0.5 point decrease in IQ. This is based on research from [Lanphear et al. (2005)](https://ehp.niehs.nih.gov/doi/10.1289/EHP5685), who find that for an individual with a BLL between 2.4 and 10 ug/dL, each 1 μg/dL increase in BLL is associated with an IQ loss of 0.51 points (95% CI: 0.32-0.70). This paper uses a spline/piecewise model to produce unique estimates of this relationship at different blood lead level ranges. The estimate we use in our model is a simplification of this approach in that we assume a constant value and therefore do not account for changes in the relationship between IQ loss and blood lead level across different blood lead levels. The average childhood blood lead levels in our focus countries fall within the 2.4 to 10 μg/dL range, hence our choice of this estimate.

In [416]:
iq_loss_per_bll_increase = pd.Series(sq.norm(0.32, 0.7) @ N)
iq_loss_per_bll_increase.head()

0   0.495
1   0.550
2   0.735
3   0.750
4   0.518
dtype: float64

#### % earnings loss per point of IQ loss

In [417]:
pct_earnings_loss_per_point_of_iq_loss = pd.Series(sq.norm(0.007, 0.024) @ N)
pct_earnings_loss_per_point_of_iq_loss.head()

0   0.017
1   0.011
2   0.007
3   0.005
4   0.010
dtype: float64

#### Earnings impact % from living in a home with lead paint

In [418]:
earnings_impact_pct_from_living_in_a_home_with_lead_paint = \
    increase_in_bll_from_living_in_a_home_with_lead_paint * \
    iq_loss_per_bll_increase * \
    pct_earnings_loss_per_point_of_iq_loss
earnings_impact_pct_from_living_in_a_home_with_lead_paint.head()

0   0.010
1   0.008
2   0.018
3   0.005
4   0.012
dtype: float64

#### Real growth in income per per capita in LMICs

In [419]:
# TODO(drw): Another independent estimate of growth? Uncertainty estimate?
real_growth_in_income_per_capita_in_lmics = 0.03

#### Real GDP per capita (PPP)

In [420]:
REAL_GDP_PER_CAPITA_PPP_INIT = pd.Series(
    [1645,
     1636,
     1782,
     1571,
     6524,
     5774,
     1309,
     2334,
     8885,
     5998,
     5432,
     5867,
     3875], index=COUNTRIES)

real_gdp_per_capita_ppp = pd.DataFrame(
    np.array(
        [REAL_GDP_PER_CAPITA_PPP_INIT * (1 + real_growth_in_income_per_capita_in_lmics)**(year - YEARS[0])
         for year in YEARS]),
    index=YEARS, columns=COUNTRIES)

real_gdp_per_capita_ppp.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
2021,"1,645.000","1,636.000","1,782.000","1,571.000","6,524.000","5,774.000","1,309.000","2,334.000","8,885.000","5,998.000","5,432.000","5,867.000","3,875.000"
2022,"1,694.350","1,685.080","1,835.460","1,618.130","6,719.720","5,947.220","1,348.270","2,404.020","9,151.550","6,177.940","5,594.960","6,043.010","3,991.250"
2023,"1,745.180","1,735.632","1,890.524","1,666.674","6,921.312","6,125.637","1,388.718","2,476.141","9,426.096","6,363.278","5,762.809","6,224.300","4,110.988"
2024,"1,797.536","1,787.701","1,947.240","1,716.674","7,128.951","6,309.406","1,430.380","2,550.425","9,708.879","6,554.177","5,935.693","6,411.029","4,234.317"
2025,"1,851.462","1,841.332","2,005.657","1,768.174","7,342.819","6,498.688","1,473.291","2,626.938","10,000.146","6,750.802","6,113.764","6,603.360","4,361.347"


#### Years until income benefit starts

In [421]:
# TODO(drw): Categorize these as inputs?
years_until_income_benefit_starts = 15

#### Years until income benefit ends

In [422]:
years_until_income_benefit_ends = 64

#### Real income improved through intervention

In [423]:
# TODO(drw): This assumes all births generate GDP per capita from 15-64, but should it account for mortality?
real_income_improved_through_intervention = pd.DataFrame(
    0.0, index=SAMPLE_YEAR_INDEX, columns=COUNTRIES)

for year in YEARS[years_until_income_benefit_starts:]:
    real_income_improved_through_intervention.loc[idx[:, year], :] = \
        births_with_averted_lead_paint_exposure_vs_counterfactual \
        .loc[idx[:, (year - years_until_income_benefit_ends):(year - years_until_income_benefit_starts)], :] \
        .groupby(level=0).sum() \
        .mul(earnings_impact_pct_from_living_in_a_home_with_lead_paint, axis=0) \
        .mul(real_gdp_per_capita_ppp.loc[year], axis=1) \
        .values

real_income_improved_through_intervention.head()
# real_income_improved_through_intervention.loc[idx[:, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi  Madagascar  Sierra Leone  Liberia  Angola  Pakistan  \
Sample Year                                                                
0      2021   0.000       0.000         0.000    0.000   0.000     0.000   
       2022   0.000       0.000         0.000    0.000   0.000     0.000   
       2023   0.000       0.000         0.000    0.000   0.000     0.000   
       2024   0.000       0.000         0.000    0.000   0.000     0.000   
       2025   0.000       0.000         0.000    0.000   0.000     0.000   

             Niger  Zimbabwe  Bolivia  Ghana  Nigeria  Cote d'Ivoire  Senegal  
Sample Year                                                                    
0      2021  0.000     0.000    0.000  0.000    0.000          0.000    0.000  
       2022  0.000     0.000    0.000  0.000    0.000          0.000    0.000  
       2023  0.000     0.000    0.000  0.000    0.000          0.000    0.000  
       2024  0.000     0.000    0.000  0.000    0.000          0.000    0.000  
       2025  0.000     0.000    0.000  0.000    0.000          0.000    0.000

#### Total discounted real income improved through intervention

In [424]:
total_discounted_real_income_improved_through_intervention = \
    real_income_improved_through_intervention.mul(discount_factor, axis=0) \
    .groupby(level=0).sum()

total_discounted_real_income_improved_through_intervention.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
Sample,,,,,,,,,,,,,
0,"1,095,649,920.966","2,420,167,623.593","342,680,213.505","380,253,454.134","9,688,858,691.912","36,130,705,354.805","906,514,393.106","1,318,062,804.221","1,642,237,359.086","2,219,770,604.536","61,959,745,566.562","8,684,772,005.124","3,260,214,835.474"
1,"415,750,828.527","769,243,115.747","143,205,246.110","175,802,320.503","4,865,021,087.842","15,513,354,236.712","324,945,058.547","508,180,720.443","1,049,218,991.215","997,065,429.614","29,558,791,512.360","3,332,587,483.625","1,201,632,509.003"
2,"2,679,669,865.280","4,079,733,634.919","644,687,372.402","734,293,154.017","25,864,131,432.522","83,118,221,985.959","1,771,297,890.792","2,511,821,286.955","4,044,146,962.210","4,679,752,834.675","156,815,853,632.720","21,406,564,187.978","5,625,895,921.659"
3,"310,765,101.280","532,021,091.700","96,316,575.105","113,976,264.677","3,778,946,313.053","12,713,119,307.307","236,699,901.251","350,939,574.585","704,410,793.598","597,739,459.781","24,706,030,113.866","2,643,135,695.786","936,622,157.570"
4,"711,520,529.784","1,456,544,465.894","226,588,446.264","268,015,175.107","8,991,900,867.225","29,100,956,679.944","510,256,827.262","700,472,107.926","1,423,275,604.401","1,644,500,623.005","54,072,527,689.553","5,537,307,212.968","2,051,909,356.154"


#### Total expected discounted real income improved through intervention

In [425]:
total_expected_discounted_real_income_improved_through_intervention = \
    total_discounted_real_income_improved_through_intervention.mul(
        probability_of_success, axis=0)

total_expected_discounted_real_income_improved_through_intervention.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
Sample,,,,,,,,,,,,,
0,"963,004,795.386","2,030,791,487.701","275,022,571.870","269,424,948.582","8,010,261,304.390","32,222,605,682.127","639,249,979.187","1,017,858,342.917","1,210,705,005.580","1,756,356,665.581","48,131,236,132.339","6,626,993,540.444","2,414,895,265.018"
1,"379,275,094.928","619,339,375.539","106,386,941.329","130,497,300.821","3,693,244,818.352","14,055,221,557.921","237,405,507.586","404,297,193.437","796,124,886.812","788,065,081.159","23,886,712,341.187","2,323,386,635.617","904,805,226.339"
2,"2,371,219,574.996","3,206,874,205.567","474,994,122.837","514,716,493.639","20,674,934,116.188","75,138,105,200.624","1,342,607,901.406","1,875,792,985.672","3,246,347,032.186","3,769,634,841.013","124,744,825,684.354","16,633,749,042.653","4,469,156,976.118"
3,"267,921,290.941","443,045,947.152","76,481,751.201","81,840,218.136","2,962,174,341.688","11,012,874,476.055","186,015,545.373","264,991,312.028","506,434,638.855","467,278,760.088","21,361,396,681.350","2,014,763,764.873","631,557,090.993"
4,"612,341,935.221","1,182,084,228.784","162,112,476.886","171,165,257.494","7,090,506,836.513","26,230,540,877.090","370,706,609.267","606,476,078.457","1,054,530,602.319","1,336,797,516.718","45,836,788,767.289","3,961,116,605.541","1,520,816,857.030"


### Health benefit

#### Annual health DALYs due to lead exposure (2019)

In [426]:
annual_health_dalys_due_to_lead_exposure_2019 = pd.DataFrame(
    np.array(
        [sq.lognorm(13896, 35975) @ N,
         sq.lognorm(18780, 71201) @ N,
         sq.lognorm(6667, 19198) @ N,
         sq.lognorm(3991, 10876) @ N,
         sq.lognorm(17490, 54720) @ N,
         sq.lognorm(411603, 982017) @ N,
         sq.lognorm(26620, 57310) @ N,
         sq.lognorm(15668, 40949) @ N,
         sq.lognorm(11090, 31897) @ N,
         sq.lognorm(6815, 47049) @ N,
         sq.lognorm(80231, 259028) @ N,
         sq.lognorm(10933, 42558) @ N,
         sq.lognorm(7114, 25100) @ N]).T,
    columns=COUNTRIES)

annual_health_dalys_due_to_lead_exposure_2019.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
0,"24,723.470","40,025.747","16,037.123","9,983.445","18,006.220","491,961.241","35,900.180","15,478.741","21,283.462","17,178.298","86,055.962","7,970.218","8,364.385"
1,"19,875.850","65,350.715","12,728.294","8,654.695","27,906.007","502,801.011","31,365.190","19,899.342","22,488.521","21,790.874","140,710.375","13,377.885","14,700.935"
2,"20,593.767","37,948.349","10,318.718","5,291.191","32,021.895","904,981.153","35,181.176","27,229.496","20,686.278","18,452.389","158,266.250","19,105.140","26,856.066"
3,"23,589.455","46,267.140","13,091.162","4,167.941","30,026.390","604,412.617","43,401.998","24,037.944","36,155.478","11,422.152","104,625.594","13,851.517","7,095.197"
4,"26,742.306","35,735.545","10,244.594","2,669.469","40,695.431","921,403.417","27,115.332","23,523.008","18,289.153","30,381.725","154,247.538","28,975.962","8,178.327"


#### Multiplier due to cardiovascular effect underestimate

In [427]:
multiplier_due_to_cardiovascular_effect_underestimate = pd.Series(
    sq.lognorm(1, 2.5) @ N)
multiplier_due_to_cardiovascular_effect_underestimate.head()

0   1.825
1   1.346
2   2.255
3   1.328
4   1.324
dtype: float64

#### Adjusted annual health DALYs due to lead exposure (2019)

In [428]:
adjusted_annual_health_dalys_due_to_lead_exposure_2019 = \
    annual_health_dalys_due_to_lead_exposure_2019.mul(
        multiplier_due_to_cardiovascular_effect_underestimate, axis=0)

adjusted_annual_health_dalys_due_to_lead_exposure_2019.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
0,"45,125.927","73,056.046","29,271.379","18,222.047","32,865.427","897,940.585","65,525.951","28,252.205","38,847.133","31,354.281","157,071.604","14,547.451","15,266.895"
1,"26,756.766","87,974.794","17,134.763","11,650.905","37,566.922","676,868.113","42,223.656","26,788.391","30,273.930","29,334.761","189,423.576","18,009.240","19,790.322"
2,"46,437.575","85,571.002","23,268.022","11,931.283","72,207.243","2,040,672.293","79,331.211","61,400.703","46,646.182","41,608.910","356,879.865","43,080.821","60,558.641"
3,"31,316.389","61,422.350","17,379.287","5,533.187","39,861.798","802,393.294","57,618.705","31,911.785","47,998.524","15,163.579","138,896.629","18,388.703","9,419.291"
4,"35,412.825","47,321.897","13,566.145","3,534.977","53,889.900","1,220,145.281","35,906.795","31,149.751","24,218.951","40,232.235","204,258.419","38,370.688","10,829.943"


#### Births (2019)

In [429]:
births_2019 = pd.Series(
    [634083,
     872971,
     261970,
     160753,
     1284366,
     6330933,
     1075960,
     484906,
     263006,
     901018,
     7697868,
     903861,
     535968], index=COUNTRIES)
births_2019.head()

Malawi           634083
Madagascar       872971
Sierra Leone     261970
Liberia          160753
Angola          1284366
dtype: int64

#### Health DALYs due to lead exposure per birth (2019)

In [430]:
health_dalys_due_to_lead_exposure_per_birth_2019 = \
    adjusted_annual_health_dalys_due_to_lead_exposure_2019.div(
        births_2019, axis=1)
health_dalys_due_to_lead_exposure_per_birth_2019.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
0,0.071,0.084,0.112,0.113,0.026,0.142,0.061,0.058,0.148,0.035,0.020,0.016,0.028
1,0.042,0.101,0.065,0.072,0.029,0.107,0.039,0.055,0.115,0.033,0.025,0.020,0.037
2,0.073,0.098,0.089,0.074,0.056,0.322,0.074,0.127,0.177,0.046,0.046,0.048,0.113
3,0.049,0.070,0.066,0.034,0.031,0.127,0.054,0.066,0.182,0.017,0.018,0.020,0.018
4,0.056,0.054,0.052,0.022,0.042,0.193,0.033,0.064,0.092,0.045,0.027,0.042,0.020


#### Average childhood BLL (2019)

In [431]:
average_childhood_bll_2019 = pd.DataFrame(
    np.array(
        [sq.lognorm(3.8, 5.6) @ N,
         sq.lognorm(3.8, 5.8) @ N,
         sq.lognorm(5, 7.6) @ N,
         sq.lognorm(5.8, 8.6) @ N,
         sq.lognorm(3.4, 5.0) @ N,
         sq.lognorm(3.9, 5.9) @ N,
         sq.lognorm(9.7, 14.5) @ N,
         sq.lognorm(7.4, 11) @ N,
         sq.lognorm(5.2, 7.8) @ N,
         sq.lognorm(2.3, 3.5) @ N,
         sq.lognorm(3.9, 5.9) @ N,
         sq.lognorm(4.1, 6.1) @ N,
         sq.lognorm(3.1, 4.7) @ N]).T, columns=COUNTRIES)

average_childhood_bll_2019.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
0,4.084,4.436,7.567,5.905,5.120,3.877,13.053,9.955,6.620,3.270,4.789,4.943,4.040
1,4.743,4.714,8.527,7.679,4.040,4.786,12.261,9.728,6.179,2.750,5.240,5.731,3.845
2,3.900,4.860,5.855,9.030,5.014,4.282,10.869,10.491,7.162,2.487,4.211,3.797,4.091
3,4.862,4.335,4.818,7.210,3.623,4.595,10.097,8.473,4.825,2.817,4.790,4.174,3.728
4,4.195,4.988,5.926,6.783,4.417,4.748,13.135,8.561,5.752,3.244,4.671,4.474,3.954


#### Health DALYs per childhood BLL

In [432]:
health_dalys_per_childhood_bll = health_dalys_due_to_lead_exposure_per_birth_2019.div(
    average_childhood_bll_2019, axis=1)
health_dalys_per_childhood_bll.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
0,0.017,0.019,0.015,0.019,0.005,0.037,0.005,0.006,0.022,0.011,0.004,0.003,0.007
1,0.009,0.021,0.008,0.009,0.007,0.022,0.003,0.006,0.019,0.012,0.005,0.003,0.010
2,0.019,0.020,0.015,0.008,0.011,0.075,0.007,0.012,0.025,0.019,0.011,0.013,0.028
3,0.010,0.016,0.014,0.005,0.009,0.028,0.005,0.008,0.038,0.006,0.004,0.005,0.005
4,0.013,0.011,0.009,0.003,0.009,0.041,0.003,0.008,0.016,0.014,0.006,0.009,0.005


#### Average years until health benefits felt

In [433]:
average_years_until_health_benefits_felt = pd.Series(sq.lognorm(15, 45) @ N)
average_years_until_health_benefits_felt.head()

0   36.303
1   38.629
2   18.477
3   13.961
4   26.854
dtype: float64

#### Health DALYs averted through intervention

In [434]:
# TODO(drw): A little confusing that this isn't computed the same way as Income benefit
health_dalys_averted_through_intervention = pd.DataFrame(
    0, index=SAMPLE_YEAR_INDEX, columns=COUNTRIES, dtype=np.float64)

for sample in SAMPLES:
    health_dalys_averted_through_intervention.loc[idx[sample, :], :] = \
        births_with_averted_lead_paint_exposure_vs_counterfactual.loc[idx[sample, :, :], :] \
        .shift(average_years_until_health_benefits_felt[sample].astype(np.int64), axis=0, fill_value=0) \
        .mul(increase_in_bll_from_living_in_a_home_with_lead_paint, level=0, axis=0) \
        .mul(health_dalys_per_childhood_bll, level=0, axis=1)

health_dalys_averted_through_intervention.head()
# health_dalys_averted_through_intervention.loc[idx[:, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi  Madagascar  Sierra Leone  Liberia  Angola  Pakistan  \
Sample Year                                                                
0      2021   0.000       0.000         0.000    0.000   0.000     0.000   
       2022   0.000       0.000         0.000    0.000   0.000     0.000   
       2023   0.000       0.000         0.000    0.000   0.000     0.000   
       2024   0.000       0.000         0.000    0.000   0.000     0.000   
       2025   0.000       0.000         0.000    0.000   0.000     0.000   

             Niger  Zimbabwe  Bolivia  Ghana  Nigeria  Cote d'Ivoire  Senegal  
Sample Year                                                                    
0      2021  0.000     0.000    0.000  0.000    0.000          0.000    0.000  
       2022  0.000     0.000    0.000  0.000    0.000          0.000    0.000  
       2023  0.000     0.000    0.000  0.000    0.000          0.000    0.000  
       2024  0.000     0.000    0.000  0.000    0.000          0.000    0.000  
       2025  0.000     0.000    0.000  0.000    0.000          0.000    0.000

#### Total discounted health DALYs averted through intervention

In [435]:
total_discounted_health_DALYs_averted_through_intervention = \
    health_dalys_averted_through_intervention.mul(
        discount_factor, axis=0).groupby(level=0).sum()

total_discounted_health_DALYs_averted_through_intervention.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
Sample,,,,,,,,,,,,,
0,"5,984.888","14,436.318","1,482.777","2,417.203","3,872.374","118,839.543","1,661.209","1,697.181","2,174.912","2,052.250","25,377.409","2,460.953","3,063.544"
1,"1,560.788","6,956.538",428.469,731.591,"3,745.232","41,576.720",550.837,851.231,"1,521.103","1,357.546","17,765.872","1,342.800","2,056.030"
2,"43,812.691","71,943.050","7,868.564","5,501.920","63,628.593","1,550,955.723","13,119.031","18,590.611","16,263.706","20,748.023","455,486.917","65,318.035","57,264.829"
3,"5,696.596","15,633.960","2,210.395","1,026.417","14,708.046","179,990.317","2,839.930","3,457.574","8,889.066","1,759.811","50,865.760","6,450.545","3,369.225"
4,"7,828.412","13,119.444","1,512.381",750.489,"17,781.258","277,669.916","1,343.969","3,045.675","3,477.028","5,102.919","76,945.344","11,989.644","3,662.212"


#### Total expected discounted health DALYs averted through intervention

In [436]:
total_expected_discounted_health_DALYs_averted_through_intervention = \
    total_discounted_health_DALYs_averted_through_intervention.mul(
        probability_of_success, axis=0)

total_expected_discounted_health_DALYs_averted_through_intervention.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
Sample,,,,,,,,,,,,,
0,"5,260.326","12,113.686","1,190.022","1,712.686","3,201.484","105,985.191","1,171.441","1,310.628","1,603.408","1,623.808","19,713.543","1,877.852","2,269.218"
1,"1,423.853","5,600.905",318.309,543.057,"2,843.165","37,668.837",402.442,677.220,"1,154.181","1,072.983","14,356.753",936.162,"1,548.149"
2,"38,769.519","56,550.827","5,797.417","3,856.673","50,862.600","1,402,049.653","9,943.959","13,883.208","13,055.320","16,712.949","362,333.493","50,754.702","45,490.623"
3,"4,911.232","13,019.338","1,755.200",737.015,"11,529.086","155,918.522","2,231.818","2,610.783","6,390.775","1,375.720","43,979.695","4,917.010","2,271.842"
4,"6,737.212","10,647.315","1,082.031",479.292,"14,021.299","250,281.534",976.407,"2,636.977","2,576.193","4,148.110","65,225.867","8,576.801","2,714.327"
